# Housing Regression Maching Learning Project

### Imports (Data and Packages)

In [130]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from data_preprocessing_final import cleaning, dummify, scale_data, ord_encoding, initiate_data
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline 

In [131]:
housing = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
housing = housing[housing['SaleCondition'] == 'Normal']
train_data_linear, train_target_linear, test_data_linear, test_target_linear, train_data_tree, train_target_tree, test_data_tree, test_target_tree = initiate_data(housing)


### Lasso Regression

In [132]:
## Working on Lasso Regression
from sklearn.linear_model import Lasso
lasso = Lasso()

In [133]:
alphaRange = np.linspace(0,0.0005,5)
params = {'alpha': alphaRange, 'max_iter' : [10000]}

In [134]:
gs = GridSearchCV(lasso, params, scoring = 'r2', cv=5)
gs.fit(train_data_linear, train_target_linear)

C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.309812260572043, tolerance: 0.020492732436991516
  model = cd_fast.enet_coordinate_descent(
C:\Users\jackc\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimat

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([0.      , 0.000125, 0.00025 , 0.000375, 0.0005  ]),
                         'max_iter': [10000]},
             scoring='r2')

In [135]:
gs.best_params_

{'alpha': 0.000125, 'max_iter': 10000}

In [136]:
gs.score(train_data_linear, train_target_linear)

0.9496018044408772

In [137]:
gs.score(test_data_linear, test_target_linear)


0.949009299604709

In [138]:
predict = gs.predict(test_data_linear)
predict = pd.DataFrame(predict)
predict.columns = ['Prediction']

## Establish Connection with Tableau


In [174]:
import tabpy_client
from tabpy.tabpy_tools.client import Client

def predict_price(_arg1,_arg2,_arg3):
    row = test_data_linear[test_data_linear['PID'] == 527164090]
    row = row.assign(TotalLivArea = _arg1)
    row = row.assign( TotRmsAbvGrd= _arg2)
    row = row.assign(GarageArea = _arg3)
    price = gs.predict(row)
    price = int(np.exp(price))
    return price

client = tabpy_client.Client('http://localhost:9004/')
client.deploy('predict_price', predict_price, 'predicts price based on inputs', override = True)

In [34]:
# final = pd.concat([predict, test_data_linear, test_target_linear], axis =1)
# pd.options.display.float_format = '${:,.2f}'.format
# final['SalePrice'] = np.exp(final['SalePrice'])
# final['Prediction'] = np.exp(final['Prediction'])
# final['Pct_diff'] = ((final['Prediction'] - final['SalePrice'])/ final['SalePrice'])*100
# final[['Prediction', 'PID','SalePrice', 'Pct_diff']]


,Prediction,PID,SalePrice,Pct_diff
0,"$131,567.89",908128050,"$137,500.00",$-4.31
1,"$134,791.43",535450070,"$133,000.00",$1.35
2,"$227,949.44",906223180,"$215,000.00",$6.02
3,"$146,896.75",909253180,"$145,000.00",$1.31
4,"$144,407.25",905103180,"$132,000.00",$9.40
...,...,...,...,...
478,"$210,358.60",903206120,"$209,500.00",$0.41
479,"$207,686.35",907290090,"$213,000.00",$-2.49
480,"$122,660.10",527450080,"$125,500.00",$-2.26
481,"$148,264.67",535402230,"$147,000.00",$0.86


## Random Forrest   

In [13]:
# rf_model = RandomForestRegressor(random_state=0)
# # rf_model.fit(train_data_tree, train_target_tree)


RandomForestRegressor(random_state=0)

In [14]:
# train_preds = rf_model.predict(train_data_tree)
# train_mse = mean_squared_error(train_target_tree, train_preds)
# print('Train set MSE is {:.3f}'.format(train_mse))
# print('Train set R2 is {:.3f}'.format(rf_model.score(train_data_tree, train_target_tree)))

# train_preds_test = rf_model.predict(test_data_tree)
# test_mse = mean_squared_error(test_target_tree, train_preds_test)
# print('Test set MSE is {:.3f}'.format(test_mse))
# print('Test set R2 is {:.3f}'.format(rf_model.score(test_data_tree, test_target_tree)))

Train set MSE is 0.002
Train set R2 is 0.987
Test set MSE is 0.014
Test set R2 is 0.900


In [15]:
# predicted = pd.DataFrame(train_preds_test, columns = ['Prediction'])

In [16]:
# final_tree = pd.concat([test_data_tree, predicted, test_target_tree], axis = 1)

In [17]:
# final_tree['SalePrice'] = np.exp(final_tree['SalePrice'])
# final_tree['Prediction'] = np.exp(final_tree['Prediction'])


In [19]:
# ## Look at which features had the most important coefficients 
# sorted(zip(train_data_tree.columns, rf_model.feature_importances_), key = lambda t: t[1], reverse=True)


[('TotalLivArea', 0.611769695940967),
 ('OverallQual', 0.08686860517170654),
 ('YearBuilt', 0.05478411049723338),
 ('ExterQual', 0.049732995132640796),
 ('GarageArea', 0.033503359751477224),
 ('LotArea', 0.01584924789713022),
 ('YearRemodAdd', 0.014954010733209075),
 ('OverallCond', 0.01378381832177892),
 ('GarageYrBlt', 0.009841980604745044),
 ('Fireplaces', 0.00858066722733795),
 ('Bsmt_Unfin_Ratio', 0.008524948559485133),
 ('KitchenQual', 0.007377373224292001),
 ('PID', 0.0066638062850456125),
 ('LotFrontage', 0.005476215459166589),
 ('Neighborhood', 0.004804537389080459),
 ('BsmtFinType1', 0.004671099787307095),
 ('GarageType', 0.0040362385248738355),
 ('MSSubClass', 0.004009441058954551),
 ('OpenPorchSF', 0.0038353831218833796),
 ('CentralAir', 0.0034308679995132658),
 ('MasVnrArea', 0.003281391509908226),
 ('MSZoning', 0.0030934202532820164),
 ('WoodDeckSF', 0.002615876766571731),
 ('MoSold', 0.0026002048637404837),
 ('EnclosedPorch', 0.0024281681202639863),
 ('BsmtQual', 0.00224